In [1]:
import pandas as pd
import re

# Read the list of filenames from the configuration file
with open('file_list.txt', 'r', encoding='utf-8') as config_file:
    file_names = config_file.read().splitlines()

# Regex pattern to match the data format
pattern = r'\[(.*?)\] (.*?): (.*)'


# Initialize an empty list to store parsed data
datalist = []
stream_count = 0
# Iterate over each specified file
for file in file_names:
    full_path = file
    with open(full_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            match = re.match(pattern, line)
            if match:
                date, user, message = match.groups()
                datalist.append([date, user, message,stream_count])
    stream_count = stream_count + 1

# Create a DataFrame from the parsed data
data = pd.DataFrame(datalist, columns=["date", "user", "message","stream"])

data['date'] = pd.to_datetime(data['date'])

In [3]:
# Group by user and stream
grouped = data.groupby(['user', 'stream'])

# Calculate the stay duration in each stream for each user
stay_duration = grouped['date'].agg(['min', 'max'])
stay_duration['duration'] = stay_duration['max'] - stay_duration['min']

# Calculate the average stay duration for each user
avg_stay_duration = stay_duration.groupby('user')['duration'].mean().reset_index()

# Rename columns for clarity
avg_stay_duration.columns = ['user', 'avg_stay_duration']

# Display the result
avg_stay_duration.describe()

,avg_stay_duration
count,5610
mean,0 days 00:17:53.194214723
std,0 days 00:38:40.814725307
min,0 days 00:00:00
25%,0 days 00:00:00
50%,0 days 00:01:19.500000
75%,0 days 00:15:32.750000
max,0 days 07:04:12


In [5]:
avg_stay_duration[avg_stay_duration['user']== 'balintboss']

,user,avg_stay_duration
2001,balintboss,0 days 04:58:46.400000
